In [1]:
# %pip install transformers
import pandas as pd
from cogdl.oag import oagbert
import torch
import re
import numpy as np
import ipywidgets as widgets
import requests
import json
from dataclasses import dataclass
from typing import Dict, List, Optional
import os

In [15]:
# Parameters
max_depth = 2
ignore_related = True
ignore_referenced = False
base_works_url = "https://api.openalex.org/works"

In [16]:
@dataclass
class Article:
    # Keeping track of some needed paper details
    id: str
    title: str
    inverted_abstract: Dict[str, List[int]]
    authors: List[str]
    host_venue: str
    affiliations: List[str]
    concepts: List[str]
    references: List[str]
    related: List[str]

    def get_abstract(self) -> str:
        abstract = dict()
        for k, v in self.inverted_abstract.items():
            for i in v:
                abstract[i] = k

        final = ""
        for i in sorted(abstract.keys()):
            final += abstract[i] + " "
        return final
    
    def fetch_references_queries(self):
        # open alex only allows 50 OR joins per request
        queries = list()
        for i in range(0, len(self.references), 50):
            queries.append('|'.join(self.references[i:i+50]))
        return queries
    
    def fetch_related_queries(self):
        # open alex only allows 50 OR joins per request
        queries = list()
        for i in range(0, len(self.related), 50):
            queries.append('|'.join(self.related[i:i+50]))
        return queries
    
    def __str__(self):
        return f"{self.id}: {self.title}\n{self.get_abstract()}"

In [17]:
def fetch_article(result):
    work_id = result["id"].split('/')[-1]
    title = result["title"]
    inverted_abstract = result['abstract_inverted_index']
    authors = [authorship['author']['display_name'] for authorship in result['authorships']]
    host_venue = result['host_venue']['publisher']
    institutions = list()

    for authorship in result['authorships']:
        for institution in authorship['institutions']: 
            if institution['display_name'] not in institutions:
                institutions.append(institution['display_name'])

    concepts = [concept['display_name'] for concept in result['concepts'] if float(concept['score']) > 0.5]
    referenced_works = [work.split('/')[-1] for work in result['referenced_works']]
    related_works = [work.split('/')[-1] for work in result['related_works']]

    return Article(
        work_id,
        title if title else "",
        inverted_abstract if inverted_abstract else {"": [0]},
        authors,
        host_venue if host_venue else "",
        institutions,
        concepts,
        referenced_works,
        related_works
    )

# Search for Article Title
Edit the title variable below to search for a paper. If not exact then returns 25 most relevant papers in the OpenAlex dataset. Select the paper in the dropdown menu.

In [58]:
title = "Black Holes"
title = title.replace(" ", "%20")
req = requests.get(base_works_url+f"?filter=title.search:{title}")
response = json.loads(req.content)

relevant_titles = [result['title'] for result in response['results']]
title_selector = widgets.Dropdown(
    options=relevant_titles,
    value=relevant_titles[0],
    description="Title: "
)
display(title_selector)

Dropdown(description='Title: ', options=('Particle creation by black holes', 'Observation of Gravitational Wav…

In [48]:
raise Exception("Please select correct title above. If done, run all cells below this one.")

Exception: Please select correct title above. If done, run all cells below this one.

In [59]:
index = relevant_titles.index(title_selector.value)
papers = dict()
root_id = response['results'][index]['id'].split('/')[-1]

papers[root_id] = fetch_article(response['results'][index])

In [60]:
use_references = ignore_referenced != True
use_related = ignore_related != True

related_works: Dict[int, List[Article]] = {}

def get_relevant_papers(current_depth: int, previous: List[Article]):
    related_works[current_depth] = []
    print(current_depth)
    for parent in previous:
        if use_references and len(parent.references) > 0:
            for query in parent.fetch_references_queries():         
                req = requests.get(base_works_url + f'?filter=openalex_id:{query}')
                res = json.loads(req.content)
                for result in res["results"]:
                    paper_id = result['id'].split('/')[-1]
                    if paper_id not in papers.keys():
                        temp = fetch_article(result)
                        papers[temp.id] = temp
                        related_works[current_depth].append(temp)
            
        if (use_related and len(parent.related) > 0) or len(parent.references) == 0:
            for query in parent.fetch_related_queries():  
                req = requests.get(base_works_url + f'?filter=openalex_id:{query}')
                res = json.loads(req.content)
                for result in res["results"]:
                    paper_id = result['id'].split('/')[-1]
                    if paper_id not in papers.keys():
                        temp = fetch_article(result)
                        papers[temp.id] = temp
                        related_works[current_depth].append(temp)

    if current_depth < max_depth:
        get_relevant_papers(current_depth+1, related_works[current_depth])

In [61]:
get_relevant_papers(1, [papers[root_id]])

1
2


In [62]:
tokenizer, model = oagbert("oagbert-v2")

In [63]:
if not os.path.exists("./embeddings/"):
    os.mkdir("./embeddings/")

files = os.listdir("./embeddings/")
files = [file.split('.')[0] for file in files]

for key in papers.keys():
    if key not in files:
        curr_paper = papers[key]
        input_ids, input_masks, token_type_ids, masked_lm_labels, position_ids, position_ids_second, masked_positions, num_spans = model.build_inputs(
            title=curr_paper.title, 
            abstract=curr_paper.get_abstract(), 
            venue=curr_paper.host_venue, 
            authors=curr_paper.authors, 
            concepts=curr_paper.concepts, 
            affiliations=curr_paper.affiliations
        )

        sequence_output, pooled_output = model.bert.forward(
            input_ids=torch.LongTensor(input_ids).unsqueeze(0),
            token_type_ids=torch.LongTensor(token_type_ids).unsqueeze(0),
            attention_mask=torch.LongTensor(input_masks).unsqueeze(0),
            output_all_encoded_layers=False,
            checkpoint_activations=False,
            position_ids=torch.LongTensor(position_ids).unsqueeze(0),
            position_ids_second=torch.LongTensor(position_ids_second).unsqueeze(0)
        )

        torch.save(pooled_output, f"./embeddings/{curr_paper.id}.pt")
        

In [64]:
root_paper_embeddings = torch.load(f"./embeddings/{root_id}.pt")
root_paper_embeddings = torch.nn.functional.normalize(root_paper_embeddings, p=2, dim=1)

paper_keys = list(papers.keys())
paper_keys.remove(root_id)

cols = ["id", "title", "score"]
similarities = pd.DataFrame(columns=cols)

for key in paper_keys:
    paper_embeddings = torch.load(f"./embeddings/{key}.pt")
    paper_embeddings = torch.nn.functional.normalize(paper_embeddings, p=2, dim=1)
    sim = torch.mm(root_paper_embeddings, paper_embeddings.transpose(0, 1))
    results = {
        "id": [key],
        "title": [papers[key].title],
        "score": [sim.detach().numpy()]
    }

    similarities = pd.concat([similarities, pd.DataFrame(results)], ignore_index=True)

In [65]:
similarities.sort_values(by="score", ascending=False).head(25)

,id,title,score
325,W2104494725,Inspiral-merger-ringdown multipolar waveforms ...,[[0.9911238]]
769,W2132442183,Improved Upper Limits on the Stochastic Gravit...,[[0.9910417]]
562,W2951135371,Prospects for Observing and Localizing Gravita...,[[0.9906782]]
597,W2063613245,Gravitational Radiation from Colliding Black H...,[[0.9904603]]
560,W2158293236,Characterization of the LIGO detectors during ...,[[0.99045527]]
857,W2047825803,Spectroscopy of Formula: the mass of the black...,[[0.9901345]]
806,W3101500586,A hierarchical method for vetoing noise transi...,[[0.9901151]]
488,W3100002341,Key elements of robustness in binary black hol...,[[0.98977697]]
225,W2118761939,Testing gravity to second post-Newtonian order...,[[0.9893247]]
179,W2044542193,Gravitational Radiation from Inspiralling Comp...,[[0.9892895]]


# Print Root Paper Abstract and 5 most similar papers

In [66]:
print(papers[root_id])

W2437571239: GW151226: Observation of Gravitational Waves from a 22-Solar-Mass Binary Black Hole Coalescence
We report the observation of a gravitational-wave signal produced by the coalescence of two stellar-mass black holes. The signal, GW151226, was observed by the twin detectors of the Laser Interferometer Gravitational-Wave Observatory (LIGO) on December 26, 2015 at 03:38:53 UTC. The signal was initially identified within 70 s by an online matched-filter search targeting binary coalescences. Subsequent off-line analyses recovered GW151226 with a network signal-to-noise ratio of 13 and a significance greater than 5 $\sigma$. The signal persisted in the LIGO frequency band for approximately 1 s, increasing in frequency and amplitude over about 55 cycles from 35 to 450 Hz, and reached a peak gravitational strain of $3.4_{-0.9}^{+0.7} \times 10^{-22}$. The inferred source-frame initial black hole masses are $14.2_{-3.7}^{+8.3} M_{\odot}$ and $7.5_{-2.3}^{+2.3} M_{\odot}$ and the final

In [67]:
ids = similarities.sort_values(by="score", ascending=False).head(5)["id"]
for id in ids.values:
    print(papers[id])
    print("\n\n")

W2104494725: Inspiral-merger-ringdown multipolar waveforms of nonspinning black-hole binaries using the effective-one-body formalism
We calibrate an effective-one-body (EOB) model to numerical-relativity simulations of mass ratios 1, 2, 3, 4, and 6, by maximizing phase and amplitude agreement of the leading (2, 2) mode and of the subleading modes (2, 1), (3, 3), (4, 4) and (5, 5). Aligning the calibrated EOB waveforms and the numerical waveforms at low frequency, the phase difference of the (2, 2) mode between model and numerical simulation remains below $\ensuremath{\sim}0.1$ rad throughout the evolution for all mass ratios considered. The fractional amplitude difference at peak amplitude of the (2, 2) mode is 2% and grows to 12% during the ringdown. Using the Advanced LIGO noise curve we study the effectualness and measurement accuracy of the EOB model, and stress the relevance of modeling the higher-order modes for parameter estimation. We find that the effectualness, measured by th